<a href="https://colab.research.google.com/github/Hhhhhxy/COGS108_Repo/blob/main/DataCheckpoint_group049.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COGS 108 - Data Checkpoint

# Names

- Cecilia Hong A15599332
- Shirui Huang A15873275
- Zifan Zhang A15928594
- Liuyi Zhao A16174128
- Xun Zhu A16575924

<a id='research_question'></a>
# Research Question

*Under the pandemic of COVID-19, is there a relationship between restaurants’ survival and their price per person, pandemic intensity, and regional income in California?*

# Dataset(s)



<b>Yelp Dataset:</b>
https://www.yelp.com/dataset/download<br>
<b>Description:</b> This dataset contains information about restaurant name, location, service, food category and whether it is closed. <br>
<b>Format:</b> JSON <br>
<b>Observations:</b> 4789 <br>

<hr>

<b>Texas COVID-19 Case Count Data by County:</b>
https://dshs.texas.gov/coronavirus/AdditionalData.aspx<br>
<b>Description:</b> This dataset contains the number of daily COVID-19 Cases from 03/04/2020 to 04/26/2021. <br>
<b>Format:</b> XLSX <br>
<b>Observations:</b> 255<br>
If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
import json
import numpy as np 
import pandas as pd 
import requests
import bs4
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
covid_xlsx = pd.read_excel("/content/drive/MyDrive/Texas COVID-19 Case Count Data by County.xlsx")
#covid = covid_xlsx.loc[:,["County Name", "Cases 04-26-2021"]]
covid_xlsx.columns = covid_xlsx.iloc[1]
covid = covid_xlsx.drop(index=[0,1,257,258,259,260,261])[["County Name", "Cases 04-26-2021"]].reset_index().drop(columns='index')
covid.columns = ['County',"Cases"]
covid.head()

,County,Cases
0,Anderson,4744
1,Andrews,1812
2,Angelina,4882
3,Aransas,949
4,Archer,685


In [ ]:
data = []
for line in open("/content/drive/MyDrive/yelp_academic_business.json", encoding="utf-8"):
    data.append(json.loads(line))
df = pd.DataFrame(data)
yelp_data = df[df['state'] == "TX"]
yelp_data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,N3_Gs3DnX4k9SgpwJxdEfw,Lane Wells Jewelry Repair,"7801 N Lamar Blvd, Ste A140",Austin,TX,78752,30.346169,-97.711458,5.0,30,1,"{'RestaurantsPriceRange2': '1', 'ByAppointment...","Shopping, Jewelry Repair, Appraisal Services, ...","{'Monday': '12:15-17:0', 'Tuesday': '12:15-17:..."
9,tXvdYGvlEceDljN8gt2_3Q,Capital City Barber Shop,"615 W Slaughter Ln, Ste 113",Austin,TX,78748,30.172706,-97.799920,4.0,5,0,"{'BusinessAcceptsCreditCards': 'False', 'Resta...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-19:0', ..."
24,nTIhpR7MhsALPwg_Hh14EA,DoubleTree by Hilton Hotel Austin,6505 N Interstate 35,Austin,TX,78752,30.326377,-97.704543,3.0,139,1,"{'WiFi': 'u'free'', 'RestaurantsPriceRange2': ...","Hotels, Hotels & Travel, Event Planning & Serv...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
27,8XyEpVdAO0o6iVkVxkWosQ,PS Property Management Company,"2506 S Lamar Blvd, Ste 2",Austin,TX,78704,30.246465,-97.778738,4.5,9,1,{'BusinessAcceptsCreditCards': 'True'},"Home Services, Real Estate, Property Management","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
37,NVfOn7TdnHbaGH97CVB_Qg,McKinley Chiropractic,"5625 Eiger Rd, Ste 160",Austin,TX,78735,30.244902,-97.857409,5.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Accept...","Chiropractors, Health & Medical","{'Monday': '9:0-17:45', 'Tuesday': '12:0-17:45..."


In [ ]:
# specify webpage we want to scrape 
wiki = 'https://en.wikipedia.org/wiki/List_of_Texas_locations_by_per_capita_income'
req = requests.get(wiki)
soup = BeautifulSoup(req.content, 'html') # get contents of web page

In [ ]:
wikitables = soup.findAll("table", 'wikitable') # get tables

In [ ]:
income_cols = []
for header in wikitables[0].find_all('tr'): # specify HTML tags
    header_name = header.find_all('th') # tag containing column names
    for head in header_name:
        income_cols.append(head.get_text()) # get the text from between the tags

In [ ]:
income = pd.DataFrame(columns=range(0,7), index = range(0,256)) # I know the size 
income.columns = [s.replace('\n','') for s in income_cols] # get rid of new line characters in column names
row_marker = -1
for row in wikitables[0].find_all('tr'):
    column_marker = 0
    columns = row.find_all('td') # different tag than above for table contents
    for column in columns:
        income.iat[row_marker,column_marker] = column.get_text().replace('\n','')
        column_marker += 1
    row_marker += 1
income.head()

,Rank,County,Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,1,Collin,"$42,220","$83,889","$84,306","782,341","181,970"
1,2,Borden,"$40,916","$58,409","$60,536",641,264
2,3,King,"$39,511","$61,563","$80,500",286,113
3,4,Loving,"$37,362","$80,504","$94,785",134,31
4,5,Kendall,"$36,418","$66,655","$80,801","33,410","12,617"


# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION